# Unit 11: Feature Understanding & Target Analysis - Practical

## 🎯 Learning Objectives

In this practical notebook, you'll learn to:
1. Analyze numerical features vs numerical targets
2. Analyze categorical features vs numerical targets
3. Analyze numerical features vs categorical targets
4. Analyze categorical features vs categorical targets
5. Rank features by importance
6. Detect feature interactions
7. Build a complete feature analysis pipeline

## 📚 Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr, f_oneway, chi2_contingency, ttest_ind
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Libraries imported successfully!")

---

## Part 1: Numerical Feature vs Numerical Target 📈

**Scenario:** Analyzing house prices

**Question:** Does house size affect price? How?

In [ ]:
# Create house price dataset
np.random.seed(42)
n_houses = 200

house_data = pd.DataFrame({
    'size_sqft': np.random.randint(800, 4000, n_houses),
    'bedrooms': np.random.randint(1, 6, n_houses),
    'age_years': np.random.randint(0, 50, n_houses),
    'distance_to_city_km': np.random.uniform(1, 30, n_houses),
    'lot_size_sqft': np.random.randint(2000, 10000, n_houses)
})

# Generate price with realistic relationships
house_data['price'] = (
    150 * house_data['size_sqft'] +  # Size effect
    20000 * house_data['bedrooms'] +  # Bedroom premium
    -3000 * house_data['age_years'] +  # Age depreciation
    -5000 * house_data['distance_to_city_km'] +  # Location penalty
    10 * house_data['lot_size_sqft'] +  # Lot size bonus
    np.random.normal(0, 50000, n_houses) +  # Random noise
    200000  # Base price
)

print("House Price Dataset:")
print(house_data.head(10))
print(f"\nShape: {house_data.shape}")
print(f"\nPrice Statistics:")
print(house_data['price'].describe())

In [ ]:
# Step 1: Visual Analysis - Scatter Plot
print("📊 VISUAL ANALYSIS: Size vs Price")
print("="*50)

plt.figure(figsize=(12, 5))

# Scatter plot
plt.subplot(1, 2, 1)
plt.scatter(house_data['size_sqft'], house_data['price'], alpha=0.6, s=50)
plt.xlabel('Size (sqft)', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.title('House Size vs Price', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Add regression line
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(house_data['size_sqft'], house_data['price'])
line = slope * house_data['size_sqft'] + intercept
plt.plot(house_data['size_sqft'], line, 'r-', linewidth=2, label=f'Trend (r={r_value:.3f})')
plt.legend()

# Hexbin plot (for density)
plt.subplot(1, 2, 2)
plt.hexbin(house_data['size_sqft'], house_data['price'], gridsize=20, cmap='YlOrRd')
plt.xlabel('Size (sqft)', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.title('Density Plot: Size vs Price', fontsize=14, fontweight='bold')
plt.colorbar(label='Count')

plt.tight_layout()
plt.show()

print("\n🔍 Visual Observations:")
print("  ✓ Clear positive relationship visible")
print("  ✓ Points cluster around trend line")
print("  ✓ Relationship appears linear")
print("  ✓ Some scatter but consistent pattern")

In [ ]:
# Step 2: Statistical Analysis
print("📈 STATISTICAL ANALYSIS")
print("="*50)

# Pearson correlation
corr_pearson, p_pearson = pearsonr(house_data['size_sqft'], house_data['price'])
print(f"\n1. Pearson Correlation:")
print(f"   Correlation: {corr_pearson:.4f}")
print(f"   P-value: {p_pearson:.6f}")
print(f"   Interpretation: {'Strong' if abs(corr_pearson) > 0.7 else 'Moderate' if abs(corr_pearson) > 0.4 else 'Weak'} positive relationship")
print(f"   Significant: {'Yes (p < 0.05)' if p_pearson < 0.05 else 'No (p >= 0.05)'}")

# Spearman correlation (rank-based)
corr_spearman, p_spearman = spearmanr(house_data['size_sqft'], house_data['price'])
print(f"\n2. Spearman Correlation:")
print(f"   Correlation: {corr_spearman:.4f}")
print(f"   Note: Similar to Pearson → relationship is linear")

# R-squared (variance explained)
from sklearn.linear_model import LinearRegression
model = LinearRegression()
X = house_data[['size_sqft']]
y = house_data['price']
model.fit(X, y)
r_squared = model.score(X, y)

print(f"\n3. R² (Variance Explained):")
print(f"   R²: {r_squared:.4f}")
print(f"   Interpretation: Size explains {r_squared*100:.1f}% of price variation")

print(f"\n4. Regression Equation:")
print(f"   Price = {model.intercept_:,.0f} + {model.coef_[0]:,.0f} × Size")
print(f"   Interpretation: Each additional sqft adds ${model.coef_[0]:,.0f} to price")

In [ ]:
# Step 3: Answering "Does this affect the target? How?"
print("\n" + "="*60)
print("🎯 ANSWER: Does house size affect price? How?")
print("="*60)

print("\n✅ YES, house size STRONGLY affects price:")
print(f"\n   📊 Correlation: {corr_pearson:.3f} (strong positive)")
print(f"   📈 Variance Explained: {r_squared*100:.1f}%")
print(f"   💰 Effect Size: +${model.coef_[0]:,.0f} per sqft")
print(f"   ✓ Statistical Significance: p < 0.001")
print(f"   ✓ Relationship Type: Linear and positive")

print("\n💡 Practical Meaning:")
print("   - Larger houses consistently cost more")
print("   - Relationship is predictable (linear)")
print("   - Size is a HIGHLY IMPORTANT feature for pricing")
print("   - Should definitely keep this feature for modeling")

print("\n📋 Feature Importance Rating: ⭐⭐⭐⭐⭐ (5/5)")

---

## Part 2: Categorical Feature vs Numerical Target 📦

**Scenario:** Product sales analysis

**Question:** Does product category affect sales amount? How?

In [ ]:
# Create sales dataset
np.random.seed(42)
n_sales = 300

sales_data = pd.DataFrame({
    'category': np.random.choice(['Electronics', 'Clothing', 'Home', 'Books', 'Sports'], n_sales),
    'region': np.random.choice(['North', 'South', 'East', 'West'], n_sales),
    'season': np.random.choice(['Spring', 'Summer', 'Fall', 'Winter'], n_sales)
})

# Generate sales amounts with category effects
category_effects = {
    'Electronics': 850,
    'Clothing': 320,
    'Home': 580,
    'Books': 180,
    'Sports': 450
}

sales_data['sales_amount'] = sales_data['category'].map(category_effects) + np.random.normal(0, 100, n_sales)

print("Sales Dataset:")
print(sales_data.head(10))
print(f"\nShape: {sales_data.shape}")
print(f"\nCategories: {sales_data['category'].unique()}")
print(f"\nSales Statistics:")
print(sales_data['sales_amount'].describe())

In [ ]:
# Step 1: Visual Analysis - Box Plot
print("📊 VISUAL ANALYSIS: Category vs Sales")
print("="*50)

plt.figure(figsize=(14, 5))

# Box plot
plt.subplot(1, 2, 1)
sns.boxplot(data=sales_data, x='category', y='sales_amount', palette='Set2')
plt.xlabel('Product Category', fontsize=12)
plt.ylabel('Sales Amount ($)', fontsize=12)
plt.title('Sales by Product Category', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')

# Violin plot (shows distribution)
plt.subplot(1, 2, 2)
sns.violinplot(data=sales_data, x='category', y='sales_amount', palette='Set2')
plt.xlabel('Product Category', fontsize=12)
plt.ylabel('Sales Amount ($)', fontsize=12)
plt.title('Sales Distribution by Category', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("\n🔍 Visual Observations:")
print("  ✓ Clear differences in median sales across categories")
print("  ✓ Electronics has highest sales")
print("  ✓ Books has lowest sales")
print("  ✓ Similar spread within categories")

In [ ]:
# Step 2: Descriptive Statistics by Category
print("📊 DESCRIPTIVE STATISTICS BY CATEGORY")
print("="*50)

category_stats = sales_data.groupby('category')['sales_amount'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).round(2)

print(category_stats)

# Calculate percentage difference from overall mean
overall_mean = sales_data['sales_amount'].mean()
category_means = sales_data.groupby('category')['sales_amount'].mean().sort_values(ascending=False)

print("\n📈 Comparison to Overall Mean ($" + f"{overall_mean:.2f})")
print("-" * 50)
for category, mean_val in category_means.items():
    pct_diff = ((mean_val - overall_mean) / overall_mean) * 100
    symbol = "📈" if pct_diff > 0 else "📉"
    print(f"{symbol} {category:15s}: ${mean_val:7.2f} ({pct_diff:+6.1f}%)")

In [ ]:
# Step 3: Statistical Testing - ANOVA
print("\n📈 STATISTICAL ANALYSIS - ANOVA")
print("="*50)

# Prepare groups for ANOVA
groups = [sales_data[sales_data['category'] == cat]['sales_amount'] 
          for cat in sales_data['category'].unique()]

# Perform ANOVA
f_statistic, p_value = f_oneway(*groups)

print(f"\nANOVA Results:")
print(f"  F-statistic: {f_statistic:.4f}")
print(f"  P-value: {p_value:.6f}")
print(f"  Significant: {'Yes (p < 0.05)' if p_value < 0.05 else 'No (p >= 0.05)'}")

if p_value < 0.05:
    print("\n  ✅ Conclusion: Categories have significantly different sales amounts")
else:
    print("\n  ❌ Conclusion: No significant difference between categories")

# Calculate effect size (eta-squared)
grand_mean = sales_data['sales_amount'].mean()
ss_between = sum([len(group) * (group.mean() - grand_mean)**2 for group in groups])
ss_total = sum([(x - grand_mean)**2 for x in sales_data['sales_amount']])
eta_squared = ss_between / ss_total

print(f"\nEffect Size (Eta-squared):")
print(f"  η² = {eta_squared:.4f}")
print(f"  Interpretation: Category explains {eta_squared*100:.1f}% of sales variance")

if eta_squared < 0.01:
    effect = "Negligible"
elif eta_squared < 0.06:
    effect = "Small"
elif eta_squared < 0.14:
    effect = "Medium"
else:
    effect = "Large"
print(f"  Effect Size: {effect}")

In [ ]:
# Step 4: Pairwise Comparisons (Post-hoc)
print("\n🔍 PAIRWISE COMPARISONS")
print("="*50)
print("\nComparing highest vs lowest categories:")

# Get highest and lowest categories
highest_cat = category_means.index[0]
lowest_cat = category_means.index[-1]

group_high = sales_data[sales_data['category'] == highest_cat]['sales_amount']
group_low = sales_data[sales_data['category'] == lowest_cat]['sales_amount']

# T-test
t_stat, p_val = ttest_ind(group_high, group_low)

print(f"\n{highest_cat} vs {lowest_cat}:")
print(f"  Mean difference: ${group_high.mean() - group_low.mean():.2f}")
print(f"  T-statistic: {t_stat:.4f}")
print(f"  P-value: {p_val:.6f}")
print(f"  Significant: {'Yes' if p_val < 0.05 else 'No'}")

# Cohen's d (effect size)
pooled_std = np.sqrt((group_high.std()**2 + group_low.std()**2) / 2)
cohens_d = (group_high.mean() - group_low.mean()) / pooled_std

print(f"\n  Cohen's d: {cohens_d:.4f}")
if abs(cohens_d) < 0.2:
    interpretation = "Small"
elif abs(cohens_d) < 0.5:
    interpretation = "Medium"
else:
    interpretation = "Large"
print(f"  Effect Size: {interpretation}")

In [ ]:
# Step 5: Answering "Does this affect the target? How?"
print("\n" + "="*60)
print("🎯 ANSWER: Does product category affect sales? How?")
print("="*60)

print("\n✅ YES, product category STRONGLY affects sales:")
print(f"\n   📊 ANOVA F-statistic: {f_statistic:.2f} (p < 0.001)")
print(f"   📈 Variance Explained: {eta_squared*100:.1f}%")
print(f"   💰 Sales Range: ${category_means.min():.0f} to ${category_means.max():.0f}")
print(f"   📉 Difference: {((category_means.max() - category_means.min()) / category_means.min() * 100):.0f}% between highest and lowest")

print("\n💡 Specific Effects:")
for i, (category, mean_val) in enumerate(category_means.items(), 1):
    print(f"   {i}. {category}: ${mean_val:.0f}")

print("\n💡 Practical Meaning:")
print("   - Category is a critical predictor of sales")
print("   - Electronics generates highest revenue")
print("   - Books generates lowest revenue")
print("   - Clear, actionable differences between categories")

print("\n📋 Feature Importance Rating: ⭐⭐⭐⭐⭐ (5/5)")

---

## Part 3: Numerical Feature vs Categorical Target 🎲

**Scenario:** Customer purchase prediction

**Question:** Does customer age affect purchase decision? How?

In [ ]:
# Create customer purchase dataset
np.random.seed(42)
n_customers = 500

customer_data = pd.DataFrame({
    'age': np.random.randint(18, 70, n_customers),
    'income': np.random.randint(20000, 150000, n_customers),
    'time_on_site_min': np.random.uniform(1, 30, n_customers)
})

# Generate purchase decision (older and higher income more likely)
purchase_prob = (
    0.01 * customer_data['age'] +  # Age effect
    0.000003 * customer_data['income'] +  # Income effect
    0.02 * customer_data['time_on_site_min'] +  # Engagement effect
    np.random.normal(0, 0.1, n_customers)
)
customer_data['purchased'] = (purchase_prob > 0.5).astype(int)
customer_data['purchased_label'] = customer_data['purchased'].map({0: 'No', 1: 'Yes'})

print("Customer Purchase Dataset:")
print(customer_data.head(10))
print(f"\nShape: {customer_data.shape}")
print(f"\nPurchase Rate: {customer_data['purchased'].mean()*100:.1f}%")
print(f"\nClass Balance:")
print(customer_data['purchased_label'].value_counts())

In [ ]:
# Step 1: Visual Analysis
print("📊 VISUAL ANALYSIS: Age vs Purchase Decision")
print("="*50)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Box plot
sns.boxplot(data=customer_data, x='purchased_label', y='age', palette='Set1', ax=axes[0])
axes[0].set_xlabel('Purchased', fontsize=12)
axes[0].set_ylabel('Age', fontsize=12)
axes[0].set_title('Age Distribution by Purchase Decision', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Histogram overlay
for purchased_val in ['No', 'Yes']:
    subset = customer_data[customer_data['purchased_label'] == purchased_val]['age']
    axes[1].hist(subset, bins=20, alpha=0.6, label=f'Purchased={purchased_val}')
axes[1].set_xlabel('Age', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Age Distribution by Purchase', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

# Violin plot
sns.violinplot(data=customer_data, x='purchased_label', y='age', palette='Set1', ax=axes[2])
axes[2].set_xlabel('Purchased', fontsize=12)
axes[2].set_ylabel('Age', fontsize=12)
axes[2].set_title('Age Distribution (Violin)', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🔍 Visual Observations:")
print("  ✓ Purchasers tend to be older (higher median)")
print("  ✓ Some overlap between groups")
print("  ✓ Distribution shapes appear similar")
print("  ✓ Clear shift in central tendency")

In [ ]:
# Step 2: Descriptive Statistics by Group
print("📊 DESCRIPTIVE STATISTICS BY PURCHASE GROUP")
print("="*50)

age_by_purchase = customer_data.groupby('purchased_label')['age'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Std', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).round(2)

print(age_by_purchase)

# Calculate difference
mean_yes = customer_data[customer_data['purchased_label'] == 'Yes']['age'].mean()
mean_no = customer_data[customer_data['purchased_label'] == 'No']['age'].mean()
age_diff = mean_yes - mean_no

print(f"\n📈 Mean Age Difference:")
print(f"  Purchased=Yes: {mean_yes:.1f} years")
print(f"  Purchased=No:  {mean_no:.1f} years")
print(f"  Difference:    {age_diff:+.1f} years")
print(f"  Percentage:    {(age_diff/mean_no)*100:+.1f}%")

In [ ]:
# Step 3: Statistical Testing - T-test
print("\n📈 STATISTICAL ANALYSIS - T-TEST")
print("="*50)

# Prepare groups
age_no = customer_data[customer_data['purchased_label'] == 'No']['age']
age_yes = customer_data[customer_data['purchased_label'] == 'Yes']['age']

# Independent samples t-test
t_statistic, p_value = ttest_ind(age_yes, age_no)

print(f"\nT-Test Results:")
print(f"  T-statistic: {t_statistic:.4f}")
print(f"  P-value: {p_value:.6f}")
print(f"  Significant: {'Yes (p < 0.05)' if p_value < 0.05 else 'No (p >= 0.05)'}")

if p_value < 0.05:
    print("\n  ✅ Conclusion: Age significantly differs between purchasers and non-purchasers")
else:
    print("\n  ❌ Conclusion: No significant age difference")

# Effect size - Cohen's d
pooled_std = np.sqrt((age_yes.std()**2 + age_no.std()**2) / 2)
cohens_d = (age_yes.mean() - age_no.mean()) / pooled_std

print(f"\nEffect Size (Cohen's d):")
print(f"  d = {cohens_d:.4f}")

if abs(cohens_d) < 0.2:
    effect = "Small"
elif abs(cohens_d) < 0.5:
    effect = "Medium"
else:
    effect = "Large"
print(f"  Effect Size: {effect}")
print(f"  Interpretation: {'Weak' if abs(cohens_d) < 0.2 else 'Moderate' if abs(cohens_d) < 0.8 else 'Strong'} practical difference")

In [ ]:
# Step 4: Point-Biserial Correlation
print("\n📈 POINT-BISERIAL CORRELATION")
print("="*50)

from scipy.stats import pointbiserialr

# Calculate correlation
corr, p_val = pointbiserialr(customer_data['purchased'], customer_data['age'])

print(f"\nCorrelation Results:")
print(f"  Correlation: {corr:.4f}")
print(f"  P-value: {p_val:.6f}")
print(f"  Interpretation: {'Positive' if corr > 0 else 'Negative'} relationship")
print(f"  Strength: {'Weak' if abs(corr) < 0.3 else 'Moderate' if abs(corr) < 0.5 else 'Strong'}")

if corr > 0:
    print("\n  → Older customers more likely to purchase")
else:
    print("\n  → Younger customers more likely to purchase")

In [ ]:
# Step 5: Threshold Analysis
print("\n🔍 THRESHOLD ANALYSIS")
print("="*50)
print("\nPurchase rate by age groups:")

# Create age bins
customer_data['age_group'] = pd.cut(customer_data['age'], 
                                     bins=[0, 30, 40, 50, 70], 
                                     labels=['18-30', '31-40', '41-50', '51+'])

# Calculate purchase rate by age group
purchase_by_age = customer_data.groupby('age_group').agg({
    'purchased': ['count', 'sum', 'mean']
}).round(3)
purchase_by_age.columns = ['Total', 'Purchases', 'Purchase_Rate']

print(purchase_by_age)

# Visualize
plt.figure(figsize=(10, 5))
purchase_rates = customer_data.groupby('age_group')['purchased'].mean() * 100
bars = plt.bar(range(len(purchase_rates)), purchase_rates, color='steelblue', alpha=0.7)
plt.xticks(range(len(purchase_rates)), purchase_rates.index)
plt.xlabel('Age Group', fontsize=12)
plt.ylabel('Purchase Rate (%)', fontsize=12)
plt.title('Purchase Rate by Age Group', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}%', ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
# Step 6: Answering "Does this affect the target? How?"
print("\n" + "="*60)
print("🎯 ANSWER: Does age affect purchase decision? How?")
print("="*60)

print("\n✅ YES, age MODERATELY affects purchase decisions:")
print(f"\n   📊 Point-Biserial Correlation: {corr:.3f}")
print(f"   📈 Mean Age Difference: {age_diff:.1f} years")
print(f"   ✓ T-test: p = {p_value:.4f} (significant)")
print(f"   ✓ Cohen's d: {cohens_d:.3f} ({effect} effect)")

print("\n💡 Specific Effects:")
for age_group in purchase_rates.index:
    rate = purchase_rates[age_group]
    print(f"   {age_group}: {rate:.1f}% purchase rate")

print("\n💡 Practical Meaning:")
print("   - Older customers are more likely to purchase")
print("   - Age provides useful predictive information")
print("   - Clear trend across age groups")
print("   - Should keep this feature for modeling")

print("\n📋 Feature Importance Rating: ⭐⭐⭐⭐ (4/5)")

---

## Part 4: Categorical Feature vs Categorical Target 🎲

**Scenario:** Customer satisfaction analysis

**Question:** Does payment method affect customer satisfaction? How?

In [ ]:
# Create satisfaction dataset
np.random.seed(42)
n_customers = 400

satisfaction_data = pd.DataFrame({
    'payment_method': np.random.choice(['Credit Card', 'PayPal', 'Bank Transfer', 'Cash'], n_customers, p=[0.4, 0.3, 0.2, 0.1]),
    'delivery_method': np.random.choice(['Standard', 'Express', 'Next Day'], n_customers),
    'customer_segment': np.random.choice(['Regular', 'Premium', 'VIP'], n_customers)
})

# Generate satisfaction with payment method effect
payment_satisfaction = {
    'Credit Card': 0.75,
    'PayPal': 0.80,
    'Bank Transfer': 0.55,
    'Cash': 0.65
}

satisfaction_prob = satisfaction_data['payment_method'].map(payment_satisfaction) + np.random.uniform(-0.2, 0.2, n_customers)
satisfaction_data['satisfied'] = (satisfaction_prob > 0.6).astype(int)
satisfaction_data['satisfied_label'] = satisfaction_data['satisfied'].map({0: 'No', 1: 'Yes'})

print("Customer Satisfaction Dataset:")
print(satisfaction_data.head(10))
print(f"\nShape: {satisfaction_data.shape}")
print(f"\nSatisfaction Rate: {satisfaction_data['satisfied'].mean()*100:.1f}%")
print(f"\nPayment Methods: {satisfaction_data['payment_method'].unique()}")

In [ ]:
# Step 1: Crosstab Analysis
print("📊 CROSSTAB ANALYSIS: Payment Method vs Satisfaction")
print("="*50)

# Create crosstab
crosstab = pd.crosstab(satisfaction_data['payment_method'], 
                       satisfaction_data['satisfied_label'],
                       margins=True)

print("\nFrequency Table:")
print(crosstab)

# Percentage crosstab
crosstab_pct = pd.crosstab(satisfaction_data['payment_method'], 
                           satisfaction_data['satisfied_label'],
                           normalize='index') * 100

print("\nPercentage Table (Row %):")
print(crosstab_pct.round(1))

In [ ]:
# Step 2: Visual Analysis
print("\n📊 VISUAL ANALYSIS")
print("="*50)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Stacked bar chart
crosstab_for_plot = pd.crosstab(satisfaction_data['payment_method'], 
                                satisfaction_data['satisfied_label'])
crosstab_for_plot.plot(kind='bar', stacked=True, ax=axes[0], color=['salmon', 'lightgreen'])
axes[0].set_xlabel('Payment Method', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Satisfaction by Payment Method (Stacked)', fontsize=13, fontweight='bold')
axes[0].legend(title='Satisfied')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
axes[0].grid(True, alpha=0.3, axis='y')

# Grouped bar chart
crosstab_for_plot.plot(kind='bar', ax=axes[1], color=['salmon', 'lightgreen'])
axes[1].set_xlabel('Payment Method', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Satisfaction by Payment Method (Grouped)', fontsize=13, fontweight='bold')
axes[1].legend(title='Satisfied')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')
axes[1].grid(True, alpha=0.3, axis='y')

# Heatmap
sns.heatmap(crosstab_for_plot, annot=True, fmt='d', cmap='YlGnBu', ax=axes[2])
axes[2].set_xlabel('Satisfied', fontsize=12)
axes[2].set_ylabel('Payment Method', fontsize=12)
axes[2].set_title('Satisfaction Heatmap', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🔍 Visual Observations:")
print("  ✓ Different satisfaction rates across payment methods")
print("  ✓ PayPal shows highest satisfaction")
print("  ✓ Bank Transfer shows lowest satisfaction")
print("  ✓ Clear visual differences")

In [ ]:
# Step 3: Chi-Square Test
print("\n📈 STATISTICAL ANALYSIS - CHI-SQUARE TEST")
print("="*50)

# Perform chi-square test
chi2, p_value, dof, expected = chi2_contingency(crosstab_for_plot)

print(f"\nChi-Square Test Results:")
print(f"  Chi-square statistic: {chi2:.4f}")
print(f"  Degrees of freedom: {dof}")
print(f"  P-value: {p_value:.6f}")
print(f"  Significant: {'Yes (p < 0.05)' if p_value < 0.05 else 'No (p >= 0.05)'}")

if p_value < 0.05:
    print("\n  ✅ Conclusion: Payment method and satisfaction are associated (not independent)")
else:
    print("\n  ❌ Conclusion: No significant association")

print("\nExpected Frequencies (if independent):")
expected_df = pd.DataFrame(expected, 
                          index=crosstab_for_plot.index,
                          columns=crosstab_for_plot.columns)
print(expected_df.round(1))

In [ ]:
# Step 4: Cramér's V (Effect Size)
print("\n📊 EFFECT SIZE - CRAMÉR'S V")
print("="*50)

def cramers_v(confusion_matrix):
    """Calculate Cramér's V statistic for categorical-categorical association"""
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    min_dim = min(confusion_matrix.shape) - 1
    return np.sqrt(chi2 / (n * min_dim))

v = cramers_v(crosstab_for_plot)

print(f"\nCramér's V:")
print(f"  V = {v:.4f}")

if v < 0.1:
    strength = "Negligible"
elif v < 0.3:
    strength = "Weak"
elif v < 0.5:
    strength = "Moderate"
else:
    strength = "Strong"

print(f"  Association Strength: {strength}")
print(f"  Interpretation: {strength} relationship between payment method and satisfaction")

In [ ]:
# Step 5: Satisfaction Rates by Payment Method
print("\n📊 SATISFACTION RATES BY PAYMENT METHOD")
print("="*50)

satisfaction_rates = satisfaction_data.groupby('payment_method').agg({
    'satisfied': ['count', 'sum', 'mean']
})
satisfaction_rates.columns = ['Total', 'Satisfied_Count', 'Satisfaction_Rate']
satisfaction_rates['Satisfaction_Rate'] = satisfaction_rates['Satisfaction_Rate'] * 100
satisfaction_rates = satisfaction_rates.sort_values('Satisfaction_Rate', ascending=False)

print(satisfaction_rates.round(1))

# Visualize satisfaction rates
plt.figure(figsize=(10, 6))
bars = plt.bar(range(len(satisfaction_rates)), satisfaction_rates['Satisfaction_Rate'], 
               color='teal', alpha=0.7)
plt.xticks(range(len(satisfaction_rates)), satisfaction_rates.index, rotation=45, ha='right')
plt.xlabel('Payment Method', fontsize=12)
plt.ylabel('Satisfaction Rate (%)', fontsize=12)
plt.title('Customer Satisfaction Rate by Payment Method', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.axhline(y=satisfaction_data['satisfied'].mean()*100, color='red', 
            linestyle='--', label=f'Overall Average ({satisfaction_data["satisfied"].mean()*100:.1f}%)')
plt.legend()

# Add value labels
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Step 6: Lift Analysis
print("\n📈 LIFT ANALYSIS")
print("="*50)

baseline_rate = satisfaction_data['satisfied'].mean()
print(f"\nBaseline Satisfaction Rate: {baseline_rate*100:.1f}%")
print("\nLift by Payment Method:")
print("-" * 40)

for payment in satisfaction_rates.index:
    rate = satisfaction_rates.loc[payment, 'Satisfaction_Rate'] / 100
    lift = rate / baseline_rate
    symbol = "📈" if lift > 1 else "📉"
    print(f"{symbol} {payment:15s}: Lift = {lift:.2f}x ({(lift-1)*100:+.1f}%)")

In [ ]:
# Step 7: Answering "Does this affect the target? How?"
print("\n" + "="*60)
print("🎯 ANSWER: Does payment method affect satisfaction? How?")
print("="*60)

print("\n✅ YES, payment method MODERATELY affects satisfaction:")
print(f"\n   📊 Chi-square: {chi2:.2f} (p = {p_value:.4f})")
print(f"   📈 Cramér's V: {v:.3f} ({strength} association)")
print(f"   💡 Satisfaction Range: {satisfaction_rates['Satisfaction_Rate'].min():.1f}% to {satisfaction_rates['Satisfaction_Rate'].max():.1f}%")

print("\n💡 Specific Effects:")
for i, (payment, row) in enumerate(satisfaction_rates.iterrows(), 1):
    print(f"   {i}. {payment}: {row['Satisfaction_Rate']:.1f}% satisfied")

print("\n💡 Practical Meaning:")
print("   - Payment method influences customer satisfaction")
print("   - PayPal users most satisfied")
print("   - Bank Transfer users least satisfied")
print("   - Consider prioritizing popular payment methods")

print("\n📋 Feature Importance Rating: ⭐⭐⭐ (3/5)")

---

## Part 5: Feature Ranking 🏆

**Scenario:** Complete dataset with multiple features

**Goal:** Rank all features by importance

In [ ]:
# Create comprehensive dataset
np.random.seed(42)
n = 500

complete_data = pd.DataFrame({
    'feature1_size': np.random.randint(500, 3000, n),
    'feature2_age': np.random.randint(1, 30, n),
    'feature3_distance': np.random.uniform(1, 50, n),
    'feature4_rooms': np.random.randint(1, 6, n),
    'feature5_random': np.random.normal(0, 1, n),  # Should be unimportant
    'feature6_category': np.random.choice(['A', 'B', 'C'], n)
})

# Generate target with known relationships
complete_data['target'] = (
    200 * complete_data['feature1_size'] +  # Strong positive
    -5000 * complete_data['feature2_age'] +  # Strong negative
    -2000 * complete_data['feature3_distance'] +  # Moderate negative
    10000 * complete_data['feature4_rooms'] +  # Moderate positive
    1000 * complete_data['feature5_random'] +  # Weak (noise)
    np.random.normal(0, 50000, n) +
    500000
)

print("Complete Dataset:")
print(complete_data.head())
print(f"\nShape: {complete_data.shape}")

In [ ]:
# Method 1: Correlation-Based Ranking
print("📊 METHOD 1: CORRELATION-BASED RANKING")
print("="*60)

# Get numerical features only
numerical_features = complete_data.select_dtypes(include=[np.number]).columns.drop('target')

# Calculate correlations
correlations = {}
for feature in numerical_features:
    corr, p_value = pearsonr(complete_data[feature], complete_data['target'])
    correlations[feature] = {'correlation': corr, 'abs_correlation': abs(corr), 'p_value': p_value}

# Create DataFrame and rank
corr_df = pd.DataFrame(correlations).T
corr_df = corr_df.sort_values('abs_correlation', ascending=False)

print("\nFeature Rankings by Correlation:")
print("-" * 60)
for rank, (feature, row) in enumerate(corr_df.iterrows(), 1):
    stars = "⭐" * min(5, int(row['abs_correlation'] * 6))
    print(f"{rank}. {feature:20s} | r = {row['correlation']:+.4f} | p = {row['p_value']:.4f} | {stars}")

# Visualize
plt.figure(figsize=(10, 6))
colors = ['green' if c > 0 else 'red' for c in corr_df['correlation']]
plt.barh(range(len(corr_df)), corr_df['correlation'], color=colors, alpha=0.7)
plt.yticks(range(len(corr_df)), corr_df.index)
plt.xlabel('Correlation with Target', fontsize=12)
plt.title('Feature Importance by Correlation', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# Method 2: R² Based Ranking
print("\n📊 METHOD 2: R² (VARIANCE EXPLAINED) RANKING")
print("="*60)

from sklearn.linear_model import LinearRegression

r2_scores = {}
for feature in numerical_features:
    model = LinearRegression()
    X = complete_data[[feature]]
    y = complete_data['target']
    model.fit(X, y)
    r2_scores[feature] = model.score(X, y)

# Rank by R²
r2_df = pd.DataFrame(r2_scores.items(), columns=['Feature', 'R²'])
r2_df = r2_df.sort_values('R²', ascending=False)

print("\nFeature Rankings by R²:")
print("-" * 60)
for rank, row in r2_df.iterrows():
    feature = row['Feature']
    r2 = row['R²']
    stars = "⭐" * min(5, int(r2 * 6))
    print(f"{rank+1}. {feature:20s} | R² = {r2:.4f} ({r2*100:.1f}% variance) | {stars}")

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(range(len(r2_df)), r2_df['R²'], color='steelblue', alpha=0.7)
plt.yticks(range(len(r2_df)), r2_df['Feature'])
plt.xlabel('R² (Variance Explained)', fontsize=12)
plt.title('Feature Importance by R²', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# Method 3: Composite Ranking
print("\n📊 METHOD 3: COMPOSITE RANKING")
print("="*60)

# Normalize scores
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

composite_scores = pd.DataFrame(index=numerical_features)
composite_scores['correlation'] = [abs(correlations[f]['correlation']) for f in numerical_features]
composite_scores['r2'] = [r2_scores[f] for f in numerical_features]
composite_scores['p_value_score'] = [1 - min(correlations[f]['p_value'], 1) for f in numerical_features]

# Normalize all scores
composite_scores_norm = pd.DataFrame(
    scaler.fit_transform(composite_scores),
    index=composite_scores.index,
    columns=composite_scores.columns
)

# Calculate weighted composite score
weights = {'correlation': 0.4, 'r2': 0.4, 'p_value_score': 0.2}
composite_scores_norm['composite'] = (
    weights['correlation'] * composite_scores_norm['correlation'] +
    weights['r2'] * composite_scores_norm['r2'] +
    weights['p_value_score'] * composite_scores_norm['p_value_score']
)

# Rank
composite_ranked = composite_scores_norm.sort_values('composite', ascending=False)

print("\nComposite Feature Rankings:")
print("-" * 70)
print(f"{'Rank':<6} {'Feature':<20} {'Composite':<12} {'Correlation':<12} {'R²':<12}")
print("-" * 70)

for rank, (feature, row) in enumerate(composite_ranked.iterrows(), 1):
    stars = "⭐" * min(5, int(row['composite'] * 6))
    print(f"{rank:<6} {feature:<20} {row['composite']:.4f}       {row['correlation']:.4f}       {row['r2']:.4f}    {stars}")

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart of composite scores
axes[0].barh(range(len(composite_ranked)), composite_ranked['composite'], color='purple', alpha=0.7)
axes[0].set_yticks(range(len(composite_ranked)))
axes[0].set_yticklabels(composite_ranked.index)
axes[0].set_xlabel('Composite Score', fontsize=12)
axes[0].set_title('Composite Feature Importance', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Heatmap of all scores
sns.heatmap(composite_ranked[['correlation', 'r2', 'p_value_score', 'composite']], 
            annot=True, fmt='.3f', cmap='YlOrRd', ax=axes[1])
axes[1].set_title('Feature Importance Heatmap', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Summary and Recommendations
print("\n" + "="*70)
print("🎯 FEATURE RANKING SUMMARY")
print("="*70)

print("\n🏆 TOP 3 MOST IMPORTANT FEATURES:")
for i, feature in enumerate(composite_ranked.index[:3], 1):
    score = composite_ranked.loc[feature, 'composite']
    corr = composite_scores.loc[feature, 'correlation']
    r2 = composite_scores.loc[feature, 'r2']
    print(f"\n{i}. {feature}")
    print(f"   Composite Score: {score:.3f}")
    print(f"   Correlation: {corr:.3f}")
    print(f"   Variance Explained: {r2*100:.1f}%")
    print(f"   ✅ KEEP - Highly important")

print("\n⚠️  LEAST IMPORTANT FEATURES:")
for i, feature in enumerate(composite_ranked.index[-2:], 1):
    score = composite_ranked.loc[feature, 'composite']
    corr = composite_scores.loc[feature, 'correlation']
    r2 = composite_scores.loc[feature, 'r2']
    print(f"\n{i}. {feature}")
    print(f"   Composite Score: {score:.3f}")
    print(f"   Correlation: {corr:.3f}")
    print(f"   Variance Explained: {r2*100:.1f}%")
    print(f"   ⚠️  CONSIDER DROPPING - Low importance")

---

## Part 6: Feature Interactions 🔗

**Goal:** Detect when features work together

In [ ]:
# Create dataset with interaction
np.random.seed(42)
n = 300

interaction_data = pd.DataFrame({
    'experience_years': np.random.randint(0, 20, n),
    'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], n),
    'city': np.random.choice(['Small', 'Medium', 'Large'], n)
})

# Map education to numeric for salary calculation
edu_map = {'High School': 0, 'Bachelor': 1, 'Master': 2, 'PhD': 3}
interaction_data['edu_numeric'] = interaction_data['education'].map(edu_map)

# Create salary with interaction: experience matters MORE for higher education
interaction_data['salary'] = (
    40000 +  # Base salary
    2000 * interaction_data['experience_years'] +  # Experience effect
    15000 * interaction_data['edu_numeric'] +  # Education effect
    1500 * interaction_data['experience_years'] * interaction_data['edu_numeric'] +  # INTERACTION!
    np.random.normal(0, 5000, n)
)

print("Salary Dataset with Interactions:")
print(interaction_data.head())
print(f"\nShape: {interaction_data.shape}")

In [ ]:
# Method 1: Visual Detection - Scatter plots by category
print("📊 VISUAL INTERACTION DETECTION")
print("="*50)

plt.figure(figsize=(12, 5))

# Plot 1: Experience vs Salary (colored by education)
plt.subplot(1, 2, 1)
for edu in ['High School', 'Bachelor', 'Master', 'PhD']:
    subset = interaction_data[interaction_data['education'] == edu]
    plt.scatter(subset['experience_years'], subset['salary'], label=edu, alpha=0.6, s=50)
    
    # Add trend line for each education level
    if len(subset) > 10:
        slope, intercept = linregress(subset['experience_years'], subset['salary'])[:2]
        x_line = np.array([subset['experience_years'].min(), subset['experience_years'].max()])
        plt.plot(x_line, slope * x_line + intercept, linewidth=2)

plt.xlabel('Experience (years)', fontsize=12)
plt.ylabel('Salary ($)', fontsize=12)
plt.title('Experience vs Salary by Education\n(Different slopes = Interaction!)', fontsize=13, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Education vs Salary (colored by experience level)
plt.subplot(1, 2, 2)
interaction_data['exp_level'] = pd.cut(interaction_data['experience_years'], 
                                       bins=[0, 5, 10, 20], 
                                       labels=['Low (0-5)', 'Med (6-10)', 'High (11+)'])

for exp_level in ['Low (0-5)', 'Med (6-10)', 'High (11+)']:
    subset = interaction_data[interaction_data['exp_level'] == exp_level]
    edu_order = ['High School', 'Bachelor', 'Master', 'PhD']
    means = [subset[subset['education'] == e]['salary'].mean() for e in edu_order]
    plt.plot(range(len(edu_order)), means, marker='o', linewidth=2, markersize=8, label=exp_level)

plt.xticks(range(len(edu_order)), edu_order, rotation=45, ha='right')
plt.xlabel('Education Level', fontsize=12)
plt.ylabel('Average Salary ($)', fontsize=12)
plt.title('Education vs Salary by Experience\n(Different slopes = Interaction!)', fontsize=13, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Observation:")
print("  ✓ Different slopes for different groups = INTERACTION")
print("  ✓ Experience effect is stronger for higher education")
print("  ✓ Education effect is stronger for more experienced workers")

In [ ]:
# Method 2: Correlation Within Groups
print("\n📊 CORRELATION ANALYSIS BY GROUP")
print("="*50)

print("\nCorrelation of Experience with Salary by Education Level:")
print("-" * 50)

for edu in ['High School', 'Bachelor', 'Master', 'PhD']:
    subset = interaction_data[interaction_data['education'] == edu]
    if len(subset) > 10:
        corr, p_value = pearsonr(subset['experience_years'], subset['salary'])
        print(f"{edu:15s}: r = {corr:+.3f} (p = {p_value:.4f})")

print("\n💡 Interpretation:")
print("  If correlations are VERY DIFFERENT across groups → INTERACTION exists")
print("  Higher education shows stronger experience-salary correlation")

In [ ]:
# Method 3: Create Interaction Feature
print("\n📊 INTERACTION FEATURE CREATION")
print("="*50)

# Create interaction term
interaction_data['experience_x_education'] = interaction_data['experience_years'] * interaction_data['edu_numeric']

# Compare correlations
corr_exp, _ = pearsonr(interaction_data['experience_years'], interaction_data['salary'])
corr_edu, _ = pearsonr(interaction_data['edu_numeric'], interaction_data['salary'])
corr_interaction, _ = pearsonr(interaction_data['experience_x_education'], interaction_data['salary'])

print("\nFeature Correlations with Salary:")
print("-" * 50)
print(f"Experience alone:        r = {corr_exp:+.4f}")
print(f"Education alone:         r = {corr_edu:+.4f}")
print(f"Experience × Education:  r = {corr_interaction:+.4f}")

print("\n💡 Interpretation:")
if abs(corr_interaction) > max(abs(corr_exp), abs(corr_edu)):
    print("  ✅ INTERACTION DETECTED!")
    print("  The interaction term has stronger correlation than individual features")
    print("  Consider adding this interaction feature to your model")
else:
    print("  ❌ No strong interaction detected")

In [ ]:
# Summary
print("\n" + "="*60)
print("🎯 INTERACTION ANALYSIS SUMMARY")
print("="*60)

print("\n✅ INTERACTION FOUND: Experience × Education")
print("\n📊 Evidence:")
print("  1. Different slopes in scatter plots by group")
print("  2. Varying correlations across education levels")
print(f"  3. Interaction term correlation ({corr_interaction:.3f}) > individual features")

print("\n💡 Practical Meaning:")
print("  - Experience matters MORE for highly educated workers")
print("  - A PhD with 10 years experience earns much more than expected")
print("  - High school graduate with 10 years earns close to expected")

print("\n📋 Recommendation:")
print("  ✅ Create interaction feature: Experience × Education")
print("  ✅ Include in modeling for better predictions")
print("  ✅ Consider other potential interactions")

---

## Part 7: Complete Feature Analysis Pipeline 🔄

**Goal:** Build a reusable function to analyze all features

In [ ]:
class FeatureAnalyzer:
    """Complete feature analysis pipeline"""
    
    def __init__(self, df, target_col):
        self.df = df
        self.target_col = target_col
        self.target = df[target_col]
        self.features = [col for col in df.columns if col != target_col]
        self.results = {}
    
    def analyze_all_features(self):
        """Analyze all features against target"""
        print("🔍 COMPLETE FEATURE ANALYSIS")
        print("="*70)
        print(f"\nDataset: {self.df.shape[0]} rows × {self.df.shape[1]} columns")
        print(f"Target: {self.target_col}")
        print(f"Features to analyze: {len(self.features)}")
        
        for feature in self.features:
            self.results[feature] = self._analyze_feature(feature)
        
        return self.results
    
    def _analyze_feature(self, feature):
        """Analyze single feature"""
        result = {
            'feature': feature,
            'type': 'numerical' if self.df[feature].dtype in ['int64', 'float64'] else 'categorical'
        }
        
        if result['type'] == 'numerical':
            # Numerical feature analysis
            corr, p_value = pearsonr(self.df[feature], self.target)
            result['correlation'] = corr
            result['abs_correlation'] = abs(corr)
            result['p_value'] = p_value
            result['significant'] = p_value < 0.05
            
            # R²
            from sklearn.linear_model import LinearRegression
            model = LinearRegression()
            X = self.df[[feature]].fillna(self.df[feature].mean())
            model.fit(X, self.target)
            result['r_squared'] = model.score(X, self.target)
            
        else:
            # Categorical feature analysis
            try:
                groups = [self.df[self.df[feature] == cat][self.target_col] 
                         for cat in self.df[feature].unique()]
                f_stat, p_value = f_oneway(*groups)
                result['f_statistic'] = f_stat
                result['p_value'] = p_value
                result['significant'] = p_value < 0.05
                
                # Effect size
                grand_mean = self.target.mean()
                ss_between = sum([len(g) * (g.mean() - grand_mean)**2 for g in groups])
                ss_total = sum([(x - grand_mean)**2 for x in self.target])
                result['eta_squared'] = ss_between / ss_total if ss_total > 0 else 0
            except:
                result['error'] = 'Could not analyze'
        
        return result
    
    def get_rankings(self):
        """Get feature importance rankings"""
        rankings = []
        
        for feature, result in self.results.items():
            if 'error' in result:
                continue
            
            if result['type'] == 'numerical':
                score = result['abs_correlation']
                metric = f"r={result['correlation']:+.3f}"
            else:
                score = result.get('eta_squared', 0)
                metric = f"η²={score:.3f}"
            
            rankings.append({
                'feature': feature,
                'type': result['type'],
                'score': score,
                'metric': metric,
                'p_value': result['p_value'],
                'significant': result['significant']
            })
        
        rankings_df = pd.DataFrame(rankings)
        rankings_df = rankings_df.sort_values('score', ascending=False)
        return rankings_df
    
    def print_report(self):
        """Print analysis report"""
        rankings = self.get_rankings()
        
        print("\n" + "="*70)
        print("📋 FEATURE ANALYSIS REPORT")
        print("="*70)
        
        print(f"\n{'Rank':<6} {'Feature':<25} {'Type':<12} {'Importance':<15} {'Significant'}")
        print("-" * 70)
        
        for rank, (idx, row) in enumerate(rankings.iterrows(), 1):
            sig_symbol = "✅" if row['significant'] else "❌"
            stars = "⭐" * min(5, int(row['score'] * 6))
            print(f"{rank:<6} {row['feature']:<25} {row['type']:<12} {row['metric']:<15} {sig_symbol}  {stars}")
        
        # Summary
        print("\n" + "="*70)
        print("📊 SUMMARY")
        print("="*70)
        
        significant_features = rankings[rankings['significant']]
        print(f"\nTotal features analyzed: {len(rankings)}")
        print(f"Significant features (p < 0.05): {len(significant_features)}")
        print(f"Non-significant features: {len(rankings) - len(significant_features)}")
        
        print("\n🏆 Top 3 Features:")
        for i, (idx, row) in enumerate(rankings.head(3).iterrows(), 1):
            print(f"  {i}. {row['feature']} ({row['metric']})")
        
        print("\n⚠️  Bottom 3 Features:")
        for i, (idx, row) in enumerate(rankings.tail(3).iterrows(), 1):
            print(f"  {i}. {row['feature']} ({row['metric']})")
    
    def plot_summary(self):
        """Create summary visualizations"""
        rankings = self.get_rankings()
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # Importance scores
        colors = ['green' if s else 'red' for s in rankings['significant']]
        axes[0].barh(range(len(rankings)), rankings['score'], color=colors, alpha=0.7)
        axes[0].set_yticks(range(len(rankings)))
        axes[0].set_yticklabels(rankings['feature'])
        axes[0].set_xlabel('Importance Score', fontsize=12)
        axes[0].set_title('Feature Importance Scores', fontsize=13, fontweight='bold')
        axes[0].grid(True, alpha=0.3, axis='x')
        
        # P-values
        axes[1].barh(range(len(rankings)), -np.log10(rankings['p_value'] + 1e-10), 
                    color=colors, alpha=0.7)
        axes[1].set_yticks(range(len(rankings)))
        axes[1].set_yticklabels(rankings['feature'])
        axes[1].set_xlabel('-log10(p-value)', fontsize=12)
        axes[1].set_title('Statistical Significance', fontsize=13, fontweight='bold')
        axes[1].axvline(x=-np.log10(0.05), color='red', linestyle='--', 
                       label='p=0.05 threshold', linewidth=2)
        axes[1].legend()
        axes[1].grid(True, alpha=0.3, axis='x')
        
        plt.tight_layout()
        plt.show()

print("✅ FeatureAnalyzer class created!")

In [ ]:
# Use the FeatureAnalyzer
print("🚀 RUNNING COMPLETE FEATURE ANALYSIS")
print("="*70)

# Create test dataset
test_data = house_data.copy()

# Initialize analyzer
analyzer = FeatureAnalyzer(test_data, 'price')

# Run analysis
results = analyzer.analyze_all_features()

# Print report
analyzer.print_report()

# Plot summary
analyzer.plot_summary()

---

## 🎯 Key Takeaways

### For Each Feature, Ask:

1. **"Does this feature affect the target?"**
   - Check statistical significance (p < 0.05)
   - Look for visual patterns
   - Consider domain knowledge

2. **"How does it affect the target?"**
   - Direction (positive/negative)
   - Strength (correlation, effect size)
   - Shape (linear, non-linear, threshold)

3. **"How important is it?"**
   - Compare with other features
   - Consider practical significance
   - Evaluate for modeling

### Analysis Methods Summary:

| Feature Type | Target Type | Visual | Statistical | Importance Metric |
|-------------|-------------|--------|-------------|-------------------|
| Numerical | Numerical | Scatter plot | Pearson r, R² | Correlation |
| Categorical | Numerical | Box plot | ANOVA, eta² | Effect size |
| Numerical | Categorical | Histogram overlay | T-test, Cohen's d | Separation |
| Categorical | Categorical | Crosstab, Heatmap | Chi², Cramér's V | Association |

---

## ✅ What You've Learned

1. ✅ Analyze numerical features vs numerical targets
2. ✅ Analyze categorical features vs numerical targets
3. ✅ Analyze numerical features vs categorical targets
4. ✅ Analyze categorical features vs categorical targets
5. ✅ Rank features by importance
6. ✅ Detect feature interactions
7. ✅ Build automated analysis pipelines

---

## 🎓 Next Steps

Ready to practice? Move on to **11_exercises.ipynb** to:
- Apply these techniques to new datasets
- Practice asking "Does this affect the target? How?"
- Build your feature analysis skills

**Happy Analyzing! 📊**